
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Professional/main/Includes/images/orders.png" width="60%">
</div>

In [0]:
%run ./Includes/Copy-Datasets-2

In [0]:

# files = dbutils.fs.ls(f"{dataset_bookstore}/kafka-raw")
# display(files)

# df_raw = spark.read.json(f"{dataset_bookstore}/kafka-raw")
# display(df_raw)

# from pyspark.sql import functions as F

# def process_bronze():
  
#     schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"

#     query = (spark.readStream
#                         .format("cloudFiles")
#                         .option("cloudFiles.format", "json")
#                         .schema(schema)
#                         .load(f"{dataset_bookstore}/kafka-raw")
#                         .withColumn("timestamp", (F.col("timestamp")/1000).cast("timestamp"))  
#                         .withColumn("year_month", F.date_format("timestamp", "yyyy-MM"))
#                   .writeStream
#                       .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/bronze")
#                       .option("mergeSchema", True) # enables schema evolution
#                       .partitionBy("topic", "year_month")
#                       .trigger(availableNow=True)
#                       .table("bronze"))
    
#     query.awaitTermination()

# process_bronze()

# batch_df = spark.table("bronze")
# display(batch_df)

# %sql
# SELECT * FROM bronze;
# -- SELECT DISTINCT(topic) FROM bronze;
# SELECT COUNT(*) FROM bronze

# bookstore.load_new_data()
# process_bronze()

In [0]:
# %sql
# SELECT cast(key AS STRING), cast(value AS STRING) FROM bronze LIMIT 20;

# SELECT v.*
# FROM (
#   SELECT from_json(cast(value AS STRING), "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>") v
#   FROM bronze
#   WHERE topic = "orders")

# (spark.readStream
#       .table("bronze")
#       .createOrReplaceTempView("bronze_tmp"))

# %sql
# CREATE OR REPLACE TEMPORARY VIEW orders_silver_tmp AS
#   SELECT v.*
#   FROM (
#     SELECT from_json(cast(value AS STRING), "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>") v
#     FROM bronze_tmp
#     WHERE topic = "orders")

# query = (spark.table("orders_silver_tmp")
#                .writeStream
#                .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/orders_silver")
#                .trigger(availableNow=True)
#                .table("orders_silver"))

# query.awaitTermination()

# from pyspark.sql import functions as F

# json_schema = "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>"

# query = (spark.readStream.table("bronze")
#         .filter("topic = 'orders'")
#         .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
#         .select("v.*")
#      .writeStream
#         .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/orders_silver")
#         .trigger(availableNow=True)
#         .table("orders_silver"))

# query.awaitTermination()


# %sql
# SELECT * FROM orders_silver;


In [0]:
# %sql
# ALTER TABLE orders_silver ADD CONSTRAINT timestamp_within_range CHECK (order_timestamp >= '2020-01-01');
# DESCRIBE EXTENDED orders_silver;

# INSERT INTO orders_silver
# VALUES ('1', '2022-02-01 00:00:00.000', 'C00001', 0, 0, NULL),
#        ('2', '2019-05-01 00:00:00.000', 'C00001', 0, 0, NULL),
#        ('3', '2023-01-01 00:00:00.000', 'C00001', 0, 0, NULL);

# SELECT *
# FROM orders_silver
# WHERE order_id IN ('1', '2', '3');

# ALTER TABLE orders_silver ADD CONSTRAINT valid_quantity CHECK (quantity > 0);

# DESCRIBE EXTENDED orders_silver;

# SELECT *
# FROM orders_silver
# where quantity <= 0;

# from pyspark.sql import functions as F

# json_schema = "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>"

# query = (spark.readStream.table("bronze")
#         .filter("topic = 'orders'")
#         .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
#         .select("v.*")
#         .filter("quantity > 0")
#      .writeStream
#         .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/orders_silver")
#         .trigger(availableNow=True)
#         .table("orders_silver"))

# query.awaitTermination()


# %sql
# ALTER TABLE orders_silver DROP CONSTRAINT timestamp_within_range;
# DESCRIBE EXTENDED orders_silver;
# DROP TABLE orders_silver;

# dbutils.fs.rm("dbfs:/mnt/demo_pro/checkpoints/orders_silver", True)

In [0]:
# (spark.read
#       .table("bronze")
#       .filter("topic = 'orders'")
#       .count()
# )

# from pyspark.sql import functions as F

# json_schema = "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>"

# batch_total = (spark.read
#                       .table("bronze")
#                       .filter("topic = 'orders'")
#                       .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
#                       .select("v.*")
#                       .dropDuplicates(["order_id", "order_timestamp"])
#                       .count()
#                 )

# print(batch_total)

# deduped_df = (spark.readStream
#                    .table("bronze")
#                    .filter("topic = 'orders'")
#                    .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
#                    .select("v.*")
#                    .withWatermark("order_timestamp", "30 seconds") ## ensures no duplicates in 30 secornd microbatch
#                    .dropDuplicates(["order_id", "order_timestamp"]))

# to make sure duplicates are not there in table
# def upsert_data(microBatchDF, batch):
#     microBatchDF.createOrReplaceTempView("orders_microbatch")
    
#     sql_query = """
#       MERGE INTO orders_silver a
#       USING orders_microbatch b
#       ON a.order_id=b.order_id AND a.order_timestamp=b.order_timestamp
#       WHEN NOT MATCHED THEN INSERT *
#     """
    
#     microBatchDF.sparkSession.sql(sql_query)
#     #microBatchDF._jdf.sparkSession().sql(sql_query) # for runtime version below 10.5

# %sql
# CREATE TABLE IF NOT EXISTS orders_silver
# (order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>);

# query = (deduped_df.writeStream
#                    .foreachBatch(upsert_data)
#                    .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/orders_silver")
#                    .trigger(availableNow=True)
#                    .start())

# query.awaitTermination()

# streaming_total = spark.read.table("orders_silver").count()
# print(f"batch total: {batch_total}")
# print(f"streaming total: {streaming_total}") # values match

In [0]:
# def type2_upsert(microBatchDF, batch):
#     microBatchDF.createOrReplaceTempView("updates")
    
#     sql_query = """
#         MERGE INTO books_silver
#         USING (
#             SELECT updates.book_id as merge_key, updates.*
#             FROM updates

#             UNION ALL

#             SELECT NULL as merge_key, updates.*
#             FROM updates
#             JOIN books_silver ON updates.book_id = books_silver.book_id
#             WHERE books_silver.current = true AND updates.price <> books_silver.price
#           ) staged_updates
#         ON books_silver.book_id = merge_key 
#         WHEN MATCHED AND books_silver.current = true AND books_silver.price <> staged_updates.price THEN
#           UPDATE SET current = false, end_date = staged_updates.updated
#         WHEN NOT MATCHED THEN
#           INSERT (book_id, title, author, price, current, effective_date, end_date)
#           VALUES (staged_updates.book_id, staged_updates.title, staged_updates.author, staged_updates.price, true, staged_updates.updated, NULL)
#     """
    
#     microBatchDF.sparkSession.sql(sql_query)

# %sql
# CREATE TABLE IF NOT EXISTS books_silver
# (book_id STRING, title STRING, author STRING, price DOUBLE, current BOOLEAN, effective_date TIMESTAMP, end_date TIMESTAMP)


# def process_books():
#     schema = "book_id STRING, title STRING, author STRING, price DOUBLE, updated TIMESTAMP"
 
#     query = (spark.readStream
#                     .table("bronze")
#                     .filter("topic = 'books'")
#                     .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#                     .select("v.*")
#                  .writeStream
#                     .foreachBatch(type2_upsert)
#                     .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/books_silver")
#                     .trigger(availableNow=True)
#                     .start()
#             )
    
#     query.awaitTermination()
    
# process_books()

# books_df = spark.read.table("books_silver").orderBy("book_id", "effective_date")
# display(books_df)

# bookstore.load_books_updates()
# bookstore.process_bronze()
# process_books()

# books_df = spark.read.table("books_silver").orderBy("book_id", "effective_date")
# display(books_df)


# %sql
# CREATE OR REPLACE TABLE current_books
# AS SELECT book_id, title, author, price
#    FROM books_silver
#    WHERE current IS TRUE;

# SELECT *
# FROM current_books
# ORDER BY book_id;



In [0]:
# from pyspark.sql import functions as F
# schema = "customer_id STRING, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country_code STRING, row_status STRING, row_time timestamp"
# customers_df = (spark.table("bronze")
#                  .filter("topic = 'customers'")
#                  .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#                  .select("v.*")
#                  .filter(F.col("row_status").isin(["insert", "update"])))
# display(customers_df)

# from pyspark.sql.window import Window
# window = Window.partitionBy("customer_id").orderBy(F.col("row_time").desc())
# ranked_df = (customers_df.withColumn("rank", F.rank().over(window))
#                           .filter("rank == 1")
#                           .drop("rank"))
# display(ranked_df)

# This will throw an exception because non-time-based window operations are not supported on streaming DataFrames.
# ranked_df = (spark.readStream
#                    .table("bronze")
#                    .filter("topic = 'customers'")
#                    .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#                    .select("v.*")
#                    .filter(F.col("row_status").isin(["insert", "update"]))
#                    .withColumn("rank", F.rank().over(window))
#                    .filter("rank == 1")
#                    .drop("rank")
#              )
# display(ranked_df)


# from pyspark.sql.window import Window
# def batch_upsert(microBatchDF, batchId):
#     window = Window.partitionBy("customer_id").orderBy(F.col("row_time").desc())
    
#     (microBatchDF.filter(F.col("row_status").isin(["insert", "update"]))
#                  .withColumn("rank", F.rank().over(window))
#                  .filter("rank == 1")
#                  .drop("rank")
#                  .createOrReplaceTempView("ranked_updates"))
    
#     query = """
#         MERGE INTO customers_silver c
#         USING ranked_updates r
#         ON c.customer_id=r.customer_id
#             WHEN MATCHED AND c.row_time < r.row_time
#               THEN UPDATE SET *
#             WHEN NOT MATCHED
#               THEN INSERT *
#     """
#     microBatchDF.sparkSession.sql(query)


# %sql
# CREATE TABLE IF NOT EXISTS customers_silver
# (customer_id STRING, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country STRING, row_time TIMESTAMP)

# df_country_lookup = spark.read.json(f"{dataset_bookstore}/country_lookup")
# display(df_country_lookup)

# query = (spark.readStream
#                   .table("bronze")
#                   .filter("topic = 'customers'")
#                   .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#                   .select("v.*")
#                   .join(F.broadcast(df_country_lookup), F.col("country_code") == F.col("code") , "inner")
#                .writeStream
#                   .foreachBatch(batch_upsert)
#                   .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/customers_silver")
#                   .trigger(availableNow=True)
#                   .start()
#           )
# query.awaitTermination()


# count = spark.table("customers_silver").count()
# expected_count = spark.table("customers_silver").select("customer_id").distinct().count()
# assert count == expected_count
# print("Unit test passed.")



In [0]:
# %sql
# ALTER TABLE customers_silver SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
# DESCRIBE TABLE EXTENDED customers_silver;
# DESCRIBE HISTORY customers_silver;

# bookstore.load_new_data()
# bookstore.process_bronze()
# bookstore.process_orders_silver()
# bookstore.process_customers_silver()

# %sql
# SELECT * FROM table_changes("customers_silver", 2);

# bookstore.load_new_data()
# bookstore.process_bronze()
# bookstore.process_orders_silver()
# bookstore.process_customers_silver()

# cdf_df = (spark.readStream
#                .format("delta")
#                .option("readChangeData", True)
#                .option("startingVersion", 2)
#                .table("customers_silver"))

# display(cdf_df)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/customers_silver")
# display(files)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/customers_silver/_change_data")
# display(files)


# bookstore.load_new_data()
# bookstore.process_bronze()
# bookstore.process_orders_silver()
# bookstore.process_customers_silver()

# cdf_df = (spark.readStream
#                .format("delta")
#                .option("readChangeData", True)
#                .option("startingVersion", 2)
#                .table("customers_silver"))

# display(cdf_df)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/customers_silver")
# display(files)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/customers_silver/_change_data")
# display(files)


In [0]:

# from pyspark.sql import functions as F
# from pyspark.sql.window import Window

# def batch_upsert(microBatchDF, batchId):
#     window = Window.partitionBy("order_id", "customer_id").orderBy(F.col("_commit_timestamp").desc())
    
#     (microBatchDF.filter(F.col("_change_type").isin(["insert", "update_postimage"]))
#                  .withColumn("rank", F.rank().over(window))
#                  .filter("rank = 1")
#                  .drop("rank", "_change_type", "_commit_version")
#                  .withColumnRenamed("_commit_timestamp", "processed_timestamp")
#                  .createOrReplaceTempView("ranked_updates"))
    
#     query = """
#         MERGE INTO customers_orders c
#         USING ranked_updates r
#         ON c.order_id=r.order_id AND c.customer_id=r.customer_id
#             WHEN MATCHED AND c.processed_timestamp < r.processed_timestamp
#               THEN UPDATE SET *
#             WHEN NOT MATCHED
#               THEN INSERT *
#     """
    
#     microBatchDF.sparkSession.sql(query)


# %sql
# CREATE TABLE IF NOT EXISTS customers_orders
# (order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country STRING, row_time TIMESTAMP, processed_timestamp TIMESTAMP)


# def process_customers_orders():
#     orders_df = spark.readStream.table("orders_silver")
    
#     cdf_customers_df = (spark.readStream
#                              .option("readChangeData", True)
#                              .option("startingVersion", 2)
#                              .table("customers_silver")
#                        )

#     query = (orders_df
#                 .join(cdf_customers_df, ["customer_id"], "inner")
#                 .writeStream
#                     .foreachBatch(batch_upsert)
#                     .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/customers_orders")
#                     .trigger(availableNow=True)
#                     .start()
#             )
    
#     query.awaitTermination()

# process_customers_orders()


# %sql
# SELECT * FROM customers_orders;

# bookstore.load_new_data()
# bookstore.process_bronze()
# bookstore.process_orders_silver()
# bookstore.process_customers_silver()
# process_customers_orders()

# %sql
# SELECT count(*) FROM customers_orders;


In [0]:
# from pyspark.sql import functions as F

# def process_books_sales():
    
#     orders_df = (spark.readStream.table("orders_silver")
#                         .withColumn("book", F.explode("books"))
#                 )

#     books_df = spark.read.table("current_books")

#     query = (orders_df
#                   .join(books_df, orders_df.book.book_id == books_df.book_id, "inner")
#                   .writeStream
#                      .outputMode("append")
#                      .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/books_sales")
#                      .trigger(availableNow=True)
#                      .table("books_sales")
#     )

#     query.awaitTermination()
    
# process_books_sales()

# %sql
# SELECT * FROM books_sales;
# SELECT count(*) FROM books_sales;

# bookstore.load_new_data()
# bookstore.process_bronze()
# bookstore.process_books_silver()
# bookstore.process_current_books()
# process_books_sales()

# %sql
# SELECT count(*) FROM books_sales;

# bookstore.process_orders_silver()
# process_books_sales()

# %sql
# SELECT count(*) FROM books_sales

In [0]:
# %sql
# CREATE VIEW IF NOT EXISTS countries_stats_vw AS (
#   SELECT country, date_trunc("DD", order_timestamp) order_date, count(order_id) orders_count, sum(quantity) books_count
#   FROM customers_orders
#   GROUP BY country, date_trunc("DD", order_timestamp)
# );

# -- execution is fast
# SELECT *
# FROM countries_stats_vw
# WHERE country = "France";

# from pyspark.sql import functions as F
# query = (spark.readStream
#                  .table("books_sales")
#                  .withWatermark("order_timestamp", "10 minutes")
#                  .groupBy(
#                      F.window("order_timestamp", "5 minutes").alias("time"),
#                      "author")
#                  .agg(
#                      F.count("order_id").alias("orders_count"),
#                      F.avg("quantity").alias ("avg_quantity"))
#               .writeStream
#                  .option("checkpointLocation", f"dbfs:/mnt/demo_pro/checkpoints/authors_stats")
#                  .trigger(availableNow=True)
#                  .table("authors_stats")
#             )
# query.awaitTermination()

# %sql
# SELECT * FROM authors_stats

In [0]:
# %sql
# DESCRIBE TABLE EXTENDED bronze;

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze")
# display(files)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/topic=customers")
# display(files)

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/topic=customers/year_month=2021-12/")
# display(files)

In [0]:
# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/_delta_log")
# display(files)

# statistics of data files are present in the add column for first 32 columns
# display(spark.read.json("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/_delta_log/00000000000000000001.json")) 

# %sql
# SELECT COUNT(*) FROM bronze;

# files = dbutils.fs.ls("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/_delta_log")
# display(files)

# display(spark.read.parquet("dbfs:/user/hive/warehouse/bookstore_eng_pro.db/bronze/_delta_log/00000000000000000010.checkpoint.parquet"))

In [0]:

## REST API
# https://docs.databricks.com/api/workspace/introduction



In [0]:
# CLI #### command line / cmd in windows
# https://docs.databricks.com/aws/en/dev-tools/cli

# pip install databricks-cli
# databricks configure --token   #### generate token from user settings in databricks
# databricks -h
# databricks clusters list
# databricks clusters start --cluster-id <id>
# databricks fs cp  dbfs:/mnt/uploads/test.json --overwrite
# databricks secrets create-scope --scope bookstore-dev
# databricks secrets put --scope bookstore-dev --key db_password --string-value 12345
# databricks secrets list --scope bookstore-dev

# %fs ls 'dbfs:/mnt/uploads'
# dbutils.secrets.help()
# db_password = dbutils.secrets.get("bookstore-dev", "db_password")
# print(db_password)


In [0]:
# #### Propagating Deletes e.g. GDPR

# from pyspark.sql import functions as F

# schema = "customer_id STRING, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country_code STRING, row_status STRING, row_time timestamp"

# (spark.readStream
#         .table("bronze")
#         .filter("topic = 'customers'")
#         .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#         .select("v.*", F.col('v.row_time').alias("request_timestamp"))
#         .filter("row_status = 'delete'")
#         .select("customer_id", "request_timestamp",
#                 F.date_add("request_timestamp", 30).alias("deadline"), 
#                 F.lit("requested").alias("status"))
#     .writeStream
#         .outputMode("append")
#         .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/delete_requests")
#         .trigger(availableNow=True)
#         .table("delete_requests")
# )

# %sql
# SELECT * FROM delete_requests;
# DELETE FROM customers_silver WHERE customer_id IN (SELECT customer_id FROM delete_requests WHERE status = 'requested');

# since the CDF was enabled
# deleteDF = (spark.readStream
#                  .format("delta")
#                  .option("readChangeFeed", "true")
#                  .option("startingVersion", 2)
#                  .table("customers_silver"))

# def process_deletes(microBatchDF, batchId):
    
#     (microBatchDF
#         .filter("_change_type = 'delete'")
#         .createOrReplaceTempView("deletes"))

#     microBatchDF._jdf.sparkSession().sql("""
#         DELETE FROM customers_orders
#         WHERE customer_id IN (SELECT customer_id FROM deletes)
#     """)
    
#     microBatchDF._jdf.sparkSession().sql("""
#         MERGE INTO delete_requests r
#         USING deletes d
#         ON d.customer_id = r.customer_id
#         WHEN MATCHED
#           THEN UPDATE SET status = "deleted"
#     """)

# (deleteDF.writeStream
#          .foreachBatch(process_deletes)
#          .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/deletes")
#          .trigger(availableNow=True)
#          .start())


# %sql
# SELECT * FROM delete_requests;
# DESCRIBE HISTORY customers_orders;

# ## Deleted records
# SELECT * FROM customers_orders@v3
# EXCEPT
# SELECT * FROM customers_orders;

# delete details present in CDF field 
# df = (spark.read
#            .option("readChangeFeed", "true")
#            .option("startingVersion", 2)
#            .table("customers_silver")
#            .filter("_change_type = 'delete'"))
# display(df)


In [0]:
%sql 
-- DESCRIBE TABLE customers_silver;

-- CREATE OR REPLACE VIEW customers_vw AS
--   SELECT
--     customer_id,
--     CASE 
--       WHEN is_member('admins_demo') THEN email
--       ELSE 'REDACTED'
--     END AS email,
--     gender,
--     CASE 
--       WHEN is_member('admins_demo') THEN first_name
--       ELSE 'REDACTED'
--     END AS first_name,
--     CASE 
--       WHEN is_member('admins_demo') THEN last_name
--       ELSE 'REDACTED'
--     END AS last_name,
--     CASE 
--       WHEN is_member('admins_demo') THEN street
--       ELSE 'REDACTED'
--     END AS street,
--     city,
--     country,
--     row_time
--   FROM customers_silver;

-- SELECT * FROM customers_vw;

-- ## Row Level Security
-- CREATE OR REPLACE VIEW customers_fr_vw AS
-- SELECT * FROM customers_vw
-- WHERE 
--   CASE 
--     WHEN is_member('admins_demo') THEN TRUE
--     ELSE country = "France" AND row_time > "2022-01-01"
--   END;

-- SELECT * FROM customers_fr_vw;


In [0]:
################## Relative Imports ###############

# %run ./helpers/cube_notebook

# c1 = Cube(3)
# c1.get_volume()

# from helpers.cube_notebook import Cube ### dont work in notebook
# from helpers.cube import Cube_PY
# c2 = Cube_PY(3)
# c2.get_volume()



In [0]:
%sh pwd


In [0]:
%sh ls /databricks/driver

In [0]:
import sys
for path in sys.path:
    print(path)

In [0]:
import os
sys.path.append(os.path.abspath('./modules'))

In [0]:
for path in sys.path:
    print(path)

In [0]:
from shapes.cube import Cube as CubeShape

In [0]:
c3 = CubeShape(3)
c3.get_volume()

In [0]:
%pip install ./wheels/shapes-1.0.0-py3-none-any.whl  # install for all nodes

In [0]:
c4 = Cube_WHL(3)
c4.get_volume()

In [0]:
%sh pip install ../wheels/shapes-1.0.0-py3-none-any.whl  # install in the driver machine only